Dependências 

In [ ]:
import os.path
import base64

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

import pandas as pd
from datetime import datetime
import time as tm
from tqdm import tqdm
from PIL import Image

from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
import base64

Funções Gerais

In [ ]:
class GmailEmail:
    def __init__(self):
        self.token_path = r"C:\Users\lucca.luna\Scripts\2 - Chaves\token_gmail.json"
        self.credentials_path = r"C:\Users\lucca.luna\Scripts\2 - Chaves\credentials_gmail.json"
        self.SCOPES = ['https://www.googleapis.com/auth/gmail.send']
        self.creds = None
        
        # Try to load credentials from token file
        if os.path.exists(self.token_path):
            self.creds = Credentials.from_authorized_user_file(self.token_path, self.SCOPES)

        # If credentials are not valid or do not exist, authenticate
        if not self.creds or not self.creds.valid:
            if self.creds and self.creds.expired and self.creds.refresh_token:
                self.creds.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(self.credentials_path, self.SCOPES)
                self.creds = flow.run_local_server(port=0)

            # Save the credentials for the next run
            with open(self.token_path, "w") as token:
                token.write(self.creds.to_json())
        
        self.service = build('gmail', 'v1', credentials=self.creds)
        self.sender_email = 'natalia.barreto@rogga.com.br'
        
    def create_message(self, to_email, subject, body):
        
        message['to'] = to_email
        message['from'] = self.sender_email
        message['subject'] = subject
        msg = MIMEText(body, 'html')
        message.attach(msg)
        raw = base64.urlsafe_b64encode(message.as_bytes()).decode()
        return {'raw': raw}
    
    
    def create_message(self, to_email, subject, body):
        message = MIMEMultipart()
        message['to'] = to_email
        message['from'] = self.sender_email
        message['subject'] = subject

        # Adiciona o corpo HTML ao e-mail
        html_part = MIMEText(body, 'html')
        message.attach(html_part)
        
        # Adiciona a imagem como anexo
        with open(r"C:\Users\lucca.luna\Scripts\7 - Arquivos\Design\datanews_banner3.png", 'rb') as img_file:
            img = MIMEImage(img_file.read())
            img.add_header('Content-ID', '<banner_image>')
            message.attach(img)

        # Codifica a mensagem em base64
        raw = base64.urlsafe_b64encode(message.as_bytes()).decode()
        return {'raw': raw}

    def send_email(self, to_email, subject, body):
        message = self.create_message(to_email, subject, body)
        try:
            message = (self.service.users().messages().send(userId='me', body=message).execute())
        except Exception as error:
            print(f"An error occurred: {error}")

Codigo

In [ ]:
#Extração de dados da planilha
arquivo_excel = r"C:\Users\lucca.luna\Scripts\4 - Scripts\0 - Sobre demanda\Newsletter\IRR.xlsx"
df = pd.read_excel(arquivo_excel)
df = df[(df['Status']  == "ATRASADO") | (df['Status']  == "PENDENTE")]
df['Data de Criação'] = pd.to_datetime(df['Data de Criação'], dayfirst=True)
df['Data Limite'] = pd.to_datetime(df['Data Limite'], dayfirst=True)


In [ ]:
from dateutil.relativedelta import relativedelta

hoje = datetime.today()
dia = hoje.weekday()
if dia == 0:
    DiaAnterior = hoje - relativedelta(days = 3)
else:
    DiaAnterior = hoje - relativedelta(days = 1)

hoje = hoje.strftime('%d/%m/%y')
DiaAnterior = DiaAnterior.strftime('%d/%m/%y')

In [ ]:
#Gera lista de emails dos compradores
df_Emails = df[['Email']]
df_Emails = df_Emails.drop_duplicates()
df_Emails = df_Emails[(df_Emails['Email'] != "") & (~df_Emails['Email'].isna())]

#Loop de envio de emails Compradores
total_compradores = len(df_Emails)
progress_bar = tqdm(total=total_compradores, desc='Enviando emails',position=0, leave=True)

for indice, linha in df_Emails.iterrows():
    
    #Filtra SubConjunto por comprador
    email_comprador = linha['Email']
    df_Resultado = df[df['Email'] == email_comprador]
    df_Resultado = df_Resultado[['ID', 'Obra', 'Categoria', 'Status', 'Descrição', 'Data de Criação', 'Data Limite', 'Dias Restantes']]
    Restricoes = len(df_Resultado)    

    #Restrições Atrasadas
    df_Atrasadas = df_Resultado[df_Resultado['Status'] == "ATRASADO"]
    Atrasadas = len(df_Atrasadas)

    #Restrições Novas
    df_Novas = df_Resultado[(df_Resultado['Data de Criação'] >= DiaAnterior)]
    Novas = len(df_Novas)

    #fru-fru datas
    df_Resultado['Data de Criação'] = df_Resultado['Data de Criação'].strftime('%d/%m/%y')
    df_Resultado['Data Limite'] = df_Resultado['Data Limite'].strftime('%d/%m/%y')

    # Adicionando o estilo diretamente no HTML
    html_template_inicio = """
    <!DOCTYPE html>
    <html>
    <head>
    <style>
    table {{
        width: 100%;
        border-collapse: collapse;
    }}
    th, td {{
        padding: 12px;
        border: 1px solid #ddd;
        text-align: left;
    }}
    th {{
        background-color: #c52126;
        color: #FFFFFF;
        font-weight: bold;
    }}
    tr:nth-child(even) {{
        background-color: #f9f9f9;
    }}
    h2 {{
        background-color: #c52126;
        color: #FFFFFF;
        padding: 10px;
        margin-bottom: 10px;
    }}
    </style>
    </head>
    <body>
    <img src="cid:banner_image" alt="Banner"><br><br>
    """

    if Novas > 0:
        html_template_Novas = """
        <h2>Restrições Novas</h2>
        {novas} <br><br>
        """
    else:
        html_template_Novas = ""

    if Atrasadas > 0:
        html_template_Atrasadas = """    
        <h2>Restrições atrasadas</h2>
        {atrasadas} <br><br>
        """
    else:
        html_template_Atrasadas = ""

    html_template_Restricoes = """
    <h2>Base Completa de Restrições</h2>
    {resultado} <br><br>
    <br> 
    """

    html_template_fim = """
    </body>
    </html>
    """

    html_template = html_template_inicio + html_template_Novas + html_template_Atrasadas + html_template_Restricoes + html_template_fim


    # Convertendo os DataFrames para HTML
    atrasadas_html = df_Atrasadas.to_html(index=False, border=0)
    resultado_html = df_Resultado.to_html(index=False, border=0)
    novas_html = df_Novas.to_html(index=False, border=0)

    # Preenchendo o template com os DataFrames
    body = html_template.format(
        atrasadas=atrasadas_html,
        resultado=resultado_html,
        novas=novas_html
    )

    if (Novas > 0) & (Atrasadas > 0):
        assunto = f"COE DATANEWS IRR - Você tem {Novas} Restrições Novas e {Atrasadas} Atrasadas"
    elif Novas > 0:
        assunto = f"COE DATANEWS IRR - Você tem {Novas} Restrições Novas"
    elif Atrasadas > 0:
        assunto = f"COE DATANEWS IRR - Você tem {Atrasadas} Restrições Atrasadas"
    else:
        assunto = f"COE DATANEWS IRR - Você tem {Restricoes} Restrições"
    

    # Criando uma instância do GmailEmail e enviando o e-mail
    gmail = GmailEmail()
    gmail.send_email(str(email_comprador), assunto, body)

    progress_bar.update(1)

    tm.sleep(1)

progress_bar.close()

In [60]:
#Gera lista de emails dos coordenadores
df_Emails = df[['Coordenador']]
df_Emails = df_Emails.drop_duplicates()
df_Emails = df_Emails[(df_Emails['Coordenador'] != "") & (~df_Emails['Coordenador'].isna())]

#Loop de envio de emails Compradores
total_compradores = len(df_Emails)
progress_bar = tqdm(total=total_compradores, desc='Enviando emails',position=0, leave=True)

for indice, linha in df_Emails.iterrows():

    # Filtra Dataframe por Coordenador
    email_coordenador = linha['Coordenador']
    df_Resultado = df[df['Coordenador'] == email_coordenador]
    df_Resultado = df_Resultado[['Responsável','ID', 'Obra', 'Categoria', 'Status', 'Descrição', 'Data de Criação', 'Data Limite', 'Dias Restantes']]
    
    # Calculando as métricas por Comprador
    Total = df_Resultado.groupby('Responsável')['ID'].count().rename('Total de Restrições')
    Atrasadas = df_Resultado[df_Resultado['Status'] == 'ATRASADO'].groupby('Responsável')['ID'].count().rename('Restrições Atrasadas')

    # Criando o DataFrame final do resumo de compradores
    df_Compradores = pd.concat([Total, Atrasadas], axis=1).fillna(0).astype({'Restrições Atrasadas': 'int'})
    df_Compradores['% De Restrições Atrasadas'] = (df_Compradores['Restrições Atrasadas'] / df_Compradores['Total de Restrições']) * 100
    df_Compradores = df_Compradores.sort_values(by='% De Restrições Atrasadas', ascending=False)
    df_Compradores['% De Restrições Atrasadas'] = df_Compradores['% De Restrições Atrasadas'].apply(lambda x: f"{x:.2f}%")

    # Resetando o índice para garantir que 'Responsável' esteja como uma coluna
    df_Compradores = df_Compradores.reset_index()
    
    # Dado para criar o titulo
    df_Atrasadas = df_Resultado[df_Resultado['Status'] == "ATRASADO"]
    Atrasadas = len(df_Atrasadas)
    
    #Restrições Novas
    df_Novas = df_Resultado[(df_Resultado['Data de Criação'] >= DiaAnterior)]
    Novas = len(df_Novas)

    #Fru-fru ordenação
    df_Resultado = df_Resultado.sort_values(by=['Responsável', 'Dias Restantes'], ascending=[False, True])

    # Adicionando o estilo diretamente no HTML
    html_template_inicio = """
    <!DOCTYPE html>
    <html>
    <head>
    <style>
    table {{
        width: 100%;
        border-collapse: collapse;
    }}
    th, td {{
        padding: 12px;
        border: 1px solid #ddd;
        text-align: left;
    }}
    th {{
        background-color: #c52126;
        color: #FFFFFF;
        font-weight: bold;
    }}
    tr:nth-child(even) {{
        background-color: #f9f9f9;
    }}
    h2 {{
        background-color: #c52126;
        color: #FFFFFF;
        padding: 10px;
        margin-bottom: 10px;
    }}
    </style>
    </head>
    <body>
    <img src="cid:banner_image" alt="Banner"><br><br>
    """


    html_template_Compradores = """
    <h2>Resumo Restrições Time</h2>
    {compradores} <br><br>
    """

    if Novas > 0:
        html_template_Novas = """
        <h2>Restrições Novas</h2>
        {novas} <br><br>
        """
    else:
        html_template_Novas = ""

    if Atrasadas > 0:
        html_template_Atrasadas = """    
        <h2>Restrições atrasadas</h2>
        {atrasadas} <br><br>
        """
    else:
        html_template_Atrasadas = ""

    html_template_Restricoes = """
    <h2>Base Completa de Restrições</h2>
    {resultado} <br><br>
    <br> 
    """

    html_template_fim = """
    </body>
    </html>
    """

    html_template = html_template_inicio + html_template_Compradores + html_template_Novas + html_template_Atrasadas + html_template_Restricoes + html_template_fim


    # Convertendo os DataFrames para HTML
    compradores_html = df_Compradores.to_html(index=False, border=0)
    atrasadas_html = df_Atrasadas.to_html(index=False, border=0)
    resultado_html = df_Resultado.to_html(index=False, border=0)
    novas_html = df_Novas.to_html(index=False, border=0)

    # Preenchendo o template com os DataFrames
    body = html_template.format(
        compradores=compradores_html,
        atrasadas=atrasadas_html,
        resultado=resultado_html,
        novas=novas_html
    )

    if (Novas > 0) & (Atrasadas > 0):
        assunto = f"COE DATANEWS IRR - Você tem {Novas} Restrições Novas e {Atrasadas} Atrasadas"
    elif Novas > 0:
        assunto = f"COE DATANEWS IRR - Você tem {Novas} Restrições Novas"
    elif Atrasadas > 0:
        assunto = f"COE DATANEWS IRR - Você tem {Atrasadas} Restrições Atrasadas"
    else:
        assunto = f"COE DATANEWS IRR - Você tem {Restricoes} Restrições"
    

    # Criando uma instância do GmailEmail e enviando o e-mail
    gmail = GmailEmail()
    gmail.send_email(str(email_coordenador), assunto, body)

    progress_bar.update(1)

    tm.sleep(1)

progress_bar.close()

Enviando emails: 100%|██████████| 7/7 [00:15<00:00,  2.26s/it]
